<a href="https://colab.research.google.com/github/gleangphibul/PDF-Data-Extraction/blob/main/PDFTextExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Setup**

In [ ]:
# For troubleshooting (no need to run)

# !pip uninstall pdfminer.six pdfminer -y

# !pip uninstall ocrmypdf pikepdf -y

In [ ]:
# Install necessary packages (Run Once)

!pip install pikepdf==10.0.2
!pip install ocrmypdf
!pip install PyMuPDF -q

! apt install tesseract-ocr
! apt install libtesseract-dev

!pip install pytesseract
!apt install ghostscript

!pip install tabula-py

!pip install PyPDF2

# Install Java (Needed for table extraction library)
!apt-get update
!apt-get install -y openjdk-11-jdk-headless

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 63 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 63 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.13).
0 upgraded, 0 newly installed, 0 to remove and 63 not upgraded.
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease

In [ ]:
# First finish setting up Java

# Set JAVA_HOME environment variable
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Verify installation
!java -version

openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)


In [ ]:
# Then import other packages

# General quality of life libraries
import requests # for general processes
import subprocess # used to run ocrmypdf
import shutil # used to replace original file with ocr-ed file
import pandas as pd # for data frame data structure, used for easy conversion to excel
import numpy as np # allows operations such as rounding
import re # Allows regular expressions, used for finding the last page of a table

import ocrmypdf # for optical character recognition

import pymupdf # used to extract text from pdf
import fitz # dependency for pymupdf

import tabula # used to extract tables

import PyPDF2 # For PDF reading and manipulation (e.g. count number of pages)


from google.colab import drive # To access files stored on google drive

In [ ]:
# Mount to Google Drive (gives code access to files in Google Drive)

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


General Helper Methods

In [ ]:
# Takes a dataframe data structure and exports it to a csv file
# Stores it in whatever file path specified (currently a google drive folder)
# Inputs: df <-- the dataframe, name <-- what you want to name the file
def export_file_to_csv(df, name):

  print("Running export_file_to_csv()")

  root_store_path = '/content/drive/My Drive/PDF_Text_Extraction_Project/Excel_Tables'
  store_path = os.path.join(root_store_path, f"{name}.csv")

  df.to_csv(store_path)

# **For Approval Form PDFs**

In [ ]:
# Fixes the misprocessing where first row becomes column names
# Grabs the first row data from the column names and make it a mergable row
# Then, return the row
def get_first_row(df):
  print("Running get_first_row()")

  # Create the first row as a list: the name, followed by empty cells
  first_row = [df.columns[0]]
  first_row.extend([""] * (df.shape[1] - 1))

  # Turn it into a mergable row (dataframe type)
  first_row_df = pd.DataFrame([first_row], columns=df.columns)

  return first_row_df


In [ ]:
# Fix second row to repeat values for subcells.
# Ex: Recruiting Agency, Recruiting Agency, Recruiting agency, Individual-New, ...
# Then, returns the row
def get_second_row(df):

  print("Running get_second_row()")
  print("Dataframe size:", df.shape)

  # Copy the values over from the second row
  second_row_copy = df.iloc[0]

  # Drop 'nan' values
  cleaned_second_row = second_row_copy.dropna()

  # Create the second row as a list
  second_row_data = cleaned_second_row.tolist()
  print(second_row_data) # Debugging statement

  # List form of the second row
  second_row_list = ["", ""]

  # Get each cell value in the original second row and write it 3 times
  # in the new second row (this one will replace the original second row)
  for value in second_row_data:
    second_row_list.extend([value] * 3)
    #second_row_list.extend([""]) # Temp line of code: ensure column sizes match

  while len(second_row_list) < df.shape[1]:
    second_row_list.extend([""])

  print("Length of second_row_list:", len(second_row_list))

  # Turn the list back into a dataframe
  second_row_df = pd.DataFrame([second_row_list], columns=df.columns)

  return second_row_df

In [ ]:

# Loops through a list of dataframes and checks for a last page
# using conditions where designated position is either: empty, 'Total', or numeric data
# inputs: a list of dataframes, a start page
def blind_find_last_page(dfs, start_page):


  page = start_page

  integer_regex = '[0-9]+'  # regular expression for any integer


  for i in range(len(dfs)):
    temp_df = dfs[i]
    pos1 = temp_df.iloc[-1, 0] # Last row, first column
    pos2 = temp_df.iloc[-1, 1] # Last row, second column
    empty_cell = pd.isna(pos1)

    # Converting data to string type to allow use of regular expressions
    if hasattr(pos2, 'decode'):
      pos2 = pos2.decode('utf-8', errors='ignore')
    else:
      pos2 = str(pos2)
    try:
      matched_regex = re.search(integer_regex, pos2)
    except:
      print("Could not use regular expression in blind_find_last_page")

    # Found the last page if the last row, first column either says 'Total' or is empty
    # Or, last row, second column displays data (integers) instead of a country name (string)
    if empty_cell or pos1 == "Total" or matched_regex:
      print("blind_find_last_page() ran successfully")
      return page
    page += 1



In [ ]:
# Manually calculates the total row by summing all the columns
# of a dataframe
# Input: a list of dataframes (each element represents a page)
def calculate_total_row(dfs):
  print("calculating total row, length of dfs is", len(dfs))

  # One page only
  if len(dfs) == 1:
    print("just one page")
    # First create a dataframe with only numerical values,
    # ready for the sum to be calculated
    df_list = []

    # Grab the rows with values only, ignoring rows that are just column titles
    # Also ignore the last row which is the total column
    first_df = dfs[0]
    temp_first_df = first_df[2:-1]
    temp_first_df.columns = range(len(temp_first_df.columns)) # Setting column names to default to avoid append conflicts
    df_list.append(temp_first_df)

  # More than one page
  else:
    print("more than one page")
    # First create a dataframe with only numerical values,
    # ready for the sum to be calculated
    df_list = []

    # Grab the rows with values only, ignoring rows that are just column titles
    first_df = dfs[0]
    temp_first_df = first_df[2:]
    temp_first_df.columns = range(len(temp_first_df.columns)) # Setting column names to default to avoid append conflicts

    df_list.append(temp_first_df)

    # For the middle pages, set the column names to default to avoid append conflicts
    # Then add it to the list ready to be appended
    for i in range(1, len(dfs) - 1):
      print("loop ran. DFS more than one page")
      temp_df = dfs[i]
      temp_df.columns = range(len(temp_df.columns))
      df_list.append(temp_df)

    # Do the same for the last page's dataframe
    last_df = dfs[-1]
    temp_last_df = last_df[:-1]
    temp_last_df.columns = range(len(temp_last_df.columns))
    df_list.append(temp_last_df)

  temp_df = pd.concat(df_list, ignore_index=True)

  last_row_list = ["", "Total"]

  # Now calculate the sum:
  for i in range(len(temp_df.columns)):
    # Third column or beyond (first two columns are id, countryname)
    if i >= 2:
      col_numeric = pd.to_numeric(temp_df.iloc[:, i], errors='coerce')
      col_sum = col_numeric.sum()
      floored_col_sum = np.floor(col_sum)
      last_row_list.append(floored_col_sum)

  # Convert list to a pandas dataframe
  last_row_df = pd.DataFrame([last_row_list], columns=range(len(temp_first_df.columns)))

  return last_row_df



In [ ]:
# For debugging
def visualize_problematic_pdf(folder_path, file_i, start_p):

  # First get the file
  files = os.listdir(folder_path)
  file = os.path.join(folder_path, files[file_i])

  # Arbitrary range of 5 pages (can be modified whenever)
  blind_start_page = start_p
  blind_end_page = blind_start_page + 5

  # Grab tables from an arbitrary page range
  temp_dfs = tabula.read_pdf(file, lattice=True, pages=f'{blind_start_page}-{blind_end_page}', silent=True)

  start_page = start_p
  last_page = blind_find_last_page(temp_dfs, start_page)

  # One page
  if start_page == last_page:
    print("one page")
    df = temp_dfs[0]
    export_file_to_csv(df, "Test")
    return df
  # More than one page
  else:
    print("more than one page")
    page_length = last_page - start_page
    dfs_list = temp_dfs[0:page_length+1]
    for i in range(len(dfs_list)):
      df = dfs_list[i]
      export_file_to_csv(df, f'test{i}')
    return dfs_list




In [ ]:
# Resizing, removing extra column on the right
def remove_extra_col(df):
  new_df = df.drop(df.columns[-1], axis=1)
  return new_df

In [ ]:
# The main function for handling problematic pdfs: pdfs that threw errors in the main function

def handle_problematic_pdf(folder_path, file_i, start_p):

  print("handle_problematic_pdf() has started")

  # First get the file
  files = os.listdir(folder_path)
  file = os.path.join(folder_path, files[file_i])

  # Arbitrary range of 5 pages (can be modified whenever)
  blind_start_page = start_p
  blind_end_page = blind_start_page + 5

  # Grab tables from an arbitrary page range
  temp_dfs = tabula.read_pdf(file, lattice=True, pages=f'{blind_start_page}-{blind_end_page}', silent=True)

  start_page = start_p
  last_page = blind_find_last_page(temp_dfs, start_page)

  #print("last_page:", last_page)

  # Check if there is an extra column on the right
  # If yes, remove it
  dfs = []

  #print("before:", len(dfs))

  for df in temp_dfs:
    num_col = df.shape[1]
    if num_col > 23:
      #print("removed rightmost column")
      new_df = remove_extra_col(df)
      dfs.append(new_df)
    else:
      #print("did not remove rightmost column")
      dfs.append(df)

  #print("after:", len(dfs))

  # Debugging statement
  #for df in dfs:
    #print(dfs)
    #print(df.shape[1])

  # Only one page
  if start_page == last_page:
    print("only one page")
    one_page_list = []
    one_page_list.append(dfs[0])
    #print(type(one_page_list[0]))

    first_row = get_first_row(one_page_list[0])
    second_row = get_second_row(one_page_list[0])

    # Last row alignment check
    pos = one_page_list[0].iloc[-1,1] # Last row, second column

    # Last row is misaligned, so realign it
    if pos != 'Total':
      print("last row is misaligned")
      print(one_page_list[0].shape)
      last_row = calculate_total_row(one_page_list)
      print("calculate_total_row() finished")
      remaining_first_table = one_page_list[0].iloc[1:-1]
      df_append_list = [first_row, second_row, remaining_first_table, last_row]

    # Last row is aligned perfectly, no changes needed
    else:
      print("last row is perfectly normal")
      remaining_first_table = one_page_list[0].iloc[1:]
      df_append_list = [first_row, second_row, remaining_first_table]

    # Standardize all df's column names to make sure appending occurs properly
    df_append_list_fixed = []
    for df in df_append_list:
      print(type(df))
      df_fixed = df.copy()
      df_fixed.columns = range(len(df_fixed.columns))
      df_append_list_fixed.append(df_fixed)

    final_df = pd.concat(df_append_list_fixed, ignore_index=True)

  # More than one page
  else:
    print("more than one page:", last_page, "pages")

    page_length = last_page - start_page
    dfs_list = dfs[0:page_length+1]
    print("Dataframes collected:", len(dfs_list))

    # Handle the first page first
    first_row = get_first_row(dfs_list[0])
    second_row = get_second_row(dfs_list[0])

    remaining_first_table = dfs_list[0].iloc[1:]

    total_row_calculate = [dfs_list[0]]

    df_append_list = [first_row, second_row, remaining_first_table]

    # Now handle all the pages up to the second last page
    integer_regex = '[0-9]+' # Used to identify whether first rows are column headers or data

    for i in range(1, len(dfs_list) - 1):
      print("entered the for loop, count:", i)

      initial_df = dfs_list[i]

      pos1 = initial_df.iloc[0, 2] # First row, second column

      # Converting data to string type to allow use of regular expressions
      if hasattr(pos1, 'decode'):
        pos1 = pos1.decode('utf-8', errors='ignore')
      else:
        pos1 = str(pos1)

      try:
        first_row_is_data = re.search(integer_regex, pos1)
      except:
        print("could not use regex in handle_problematic_pdf()")

      # If first rows are column headers instead of data
      # Grab only the data rows
      if not first_row_is_data:
        print("first row is not data, column headers!")
        df = initial_df.iloc[2:]

        df_append_list.append(fixed_df)
        total_row_calculate.append(fixed_df)
      else:
        print("first row is data")
        df = dfs_list[i]

        # Here the first row accidentally gets treated as a header
        # Fix this by grabbing data from the header and inserting it as
        # the first row
        header_data = df.columns.tolist()
        header_df = pd.DataFrame([header_data], columns=df.columns) # Turn it back into a dataframe

        fixed_df = pd.concat([header_df, df], ignore_index=True)

        df_append_list.append(fixed_df)
        total_row_calculate.append(fixed_df)

    # For the last page
    print("for loop finished running")

    initial_last_page = dfs_list[-1]
    pos1 = initial_last_page.iloc[0, 2] # First row, second column

    # Converting data to string type to allow use of regular expressions
    if hasattr(pos1, 'decode'):
      pos1 = pos1.decode('utf-8', errors='ignore')
    else:
      pos1 = str(pos1)

    first_row_is_data = re.search(integer_regex, pos1)

    if not first_row_is_data:
      print("first row is not data, column headers!")
      fixed_last_page = initial_last_page.iloc[2:]
    else:
      page_last = dfs_list[-1]

      # First row was mistakenly taken as a column name so add that back into the dataframe
      last_p_header_data = page_last.columns.tolist()
      last_p_header_df = pd.DataFrame([last_p_header_data], columns=page_last.columns)

      fixed_last_page = pd.concat([last_p_header_df, page_last], ignore_index=True)

    last_page_upper = fixed_last_page.iloc[:-1]
    df_append_list.append(last_page_upper)

    total_row_calculate.append(fixed_last_page)

    # Now for the last page, we need to manually calculate the total row ourselves
    last_row = calculate_total_row(total_row_calculate)
    #print(last_row)

    df_append_list.append(last_row)

    # Standardize all df's column names to make sure appending occurs properly
    df_append_list_fixed = []
    for df in df_append_list:
      df_fixed = df.copy()
      df_fixed.columns = range(len(df_fixed.columns))
      df_append_list_fixed.append(df_fixed)

    # Once everything (e.g. pages) is ordered correctly, append the separate dataframes
    final_df = pd.concat(df_append_list_fixed, ignore_index=True)
    #print(final_df)

  print("handle_problematic_pdf() has finished running")
  #return final_df
  return final_df, last_page


Main Function

In [ ]:
# Loop Function

def get_approval_PDFs(folder_p):

  # List of data frames to append later
  country_dfs = []  # for countrywise
  district_dfs = [] # for districtwise

  # Gets all the files in a given folder
  files = os.listdir(folder_p)

  # If folder is not empty
  if files:
    # i now acts as the file index in the folder
    for i in range(len(files)):
      print("Processing file:", files[i])

      try:
        # Country-wise
        print("Running handle_problematic_pdf (country) on", files[i])
        start_page = 1
        dfs_page = handle_problematic_pdf(folder_p, i, start_page)

        print("Returned output from handle_problematic_pdf")

        country_df = dfs_page[0]
        last_page = dfs_page[1]

        # Add a time stamp column (to the leftmost column)
        temp_file_name = files[i].split('.')
        timestamp_val = temp_file_name[0]

        if i == 0:
          timestamp_col = ["", "", ""]
          for _ in range(country_df.shape[0] - 3):
            timestamp_col.append(timestamp_val)
          #print(len(timestamp_col))
          #print(len(country_df))
          country_df.insert(loc=0, column='Timestamp', value=timestamp_col)
        else:
          country_df.insert(loc=0, column='Timestamp', value=timestamp_val)

        # If not the first file, delete the header rows (first three rows)
        if i > 0:
          country_df = country_df.iloc[3:]

        country_dfs.append(country_df)

        start_page = last_page + 1

        # District-wise
        print("Running handle_problematic_pdf (district) on", files[i])
        dfs_page = handle_problematic_pdf(folder_p, i, start_page)
        district_df = dfs_page[0]

        # Add a time stamp column (to the leftmost column)
        if i == 0:
          timestamp_col = ["", "", ""]
          for _ in range(district_df.shape[0] - 3):
            timestamp_col.append(timestamp_val)
          #print(len(timestamp_col))
          #print(len(country_df))
          district_df.insert(loc=0, column='Timestamp', value=timestamp_col)
        else:
          district_df.insert(loc=0, column='Timestamp', value=timestamp_val)

        # If not the first file, delete the header rows (first three rows)
        if i > 0:
          district_df = district_df.iloc[3:]

        district_dfs.append(district_df)
      except:
        print("Error occured on file:", files[i])

  # Append files
  final_countrywise_df = pd.concat(country_dfs, ignore_index=True)
  final_districtwise_df = pd.concat(district_dfs, ignore_index=True)

  # Export file to csv
  export_file_to_csv(final_countrywise_df, "countrywise")
  export_file_to_csv(final_districtwise_df, "districtwise")



Testing

In [ ]:
folder_path = '/content/drive/My Drive/PDF_Text_Extraction_Project/Approval_PDFs'
#folder_path = '/content/drive/My Drive/PDF_Text_Extraction_Project/TestPdf'
#folder_path = '/content/drive/My Drive/PDF_Text_Extraction_Project/Problematic_PDFS'

test = get_approval_PDFs(folder_path)

Processing file: 2025_05_14.pdf
Running handle_problematic_pdf (country) on 2025_05_14.pdf
handle_problematic_pdf() has started
blind_find_last_page() ran successfully
more than one page: 2 pages
Dataframes collected: 2
Running get_first_row()
Running get_second_row()
Dataframe size: (89, 23)
['Recruiting Agency', 'Individual-New', 'G-to-G', 'Individual-ReEntry', 'Legalization', 'Total with ReEntry', 'Total without ReEntry']
Length of second_row_list: 23
for loop finished running
first row is not data, column headers!
calculating total row, length of dfs is 2
more than one page
handle_problematic_pdf() has finished running
Returned output from handle_problematic_pdf
Running handle_problematic_pdf (district) on 2025_05_14.pdf
handle_problematic_pdf() has started
blind_find_last_page() ran successfully
only one page
Running get_first_row()
Running get_second_row()
Dataframe size: (80, 23)
['Recruiting', 'Individual-New', 'G-to-G', 'Individual-ReEntry', 'Legalization', 'Total with ReEntry

# **For Departure/Arrival Form PDFs:**

In [ ]:
# Gets the last page number of the departure/arrival form
# Uses the PyPDF2 library which stores number of pages by default
def get_DA_last_page_num(file):

  my_file = open(file, 'rb')
  pdfReader = PyPDF2.PdfReader(file)
  totalPages = len(pdfReader.pages)

  last_page_num = totalPages
  #print(last_page_num)
  return last_page_num


In [ ]:
# Extracts tables from each page as a pandas dataframe
# Uses get_DA_last_page_num to determine the last page number
# returns the list of the dataframes (each element is each page's table)
def extract_DA_forms(file):

  first_page = 1
  last_page = get_DA_last_page_num(file)

  dfs = tabula.read_pdf(file, lattice=True, pages=f'{first_page}-{last_page}', silent=True)

  return dfs

In [ ]:
# If normal table extraction not working, try different setting: stream
def extract_DA_forms_stream(file):

  first_page = 1
  last_page = get_DA_last_page_num(file)

  dfs = tabula.read_pdf(file, stream=True, pages=f'{first_page}-{last_page}', silent=True)

  return dfs


In [ ]:
# Gets the first row
# Hardcodes the values
def get_DA_first_row(df):

  first_row_list = ["", "", "Arrival", "Arrival", "Arrival", "Arrival", "Departure", "Departure", "Departure", "Departure"]

  first_row_df = pd.DataFrame([first_row_list], columns=df.columns)

  return first_row_df

In [ ]:
# Gets the second row
# Hard codes the values
def get_DA_second_row(df):

  second_row_list = ["S.N.", "Country", "F", "M", "Unverified", "Total", "F", "M", "Unverified", "Total"]

  second_row_df = pd.DataFrame([second_row_list], columns=df.columns)

  return second_row_df

In [ ]:
# Removes completely empty columns and returns the modified dataframe
def remove_empty_columns(df):
  print("called remove_empty_columns()")

  df_cleaned = df.dropna(axis=1, how='all')

  return df_cleaned

In [ ]:
def add_missing_total_col(df):
  print("called add_missing_total_col()")

  total_column = []

  for i in range(0, df.shape[0]):
    try:
      col1 = int(df.iloc[i, 6])
      col2 = int(df.iloc[i, 7])
      col3 = int(df.iloc[i, 8])
      #print(col1, col2, col3)
      total_val = col1 + col2 + col3
      #print(total_val)
      total_column.append(total_val)
    except:
      print("could not add values across row", i)
      total_column.append(0)

  #print(len(total_column))

  #new_column = pd.DataFrame(total_column)

  #df.insert(9, "totalcol", total_column)
  #print(len(df))
  #print(len(total_column))
  #df["totalcol"] = total_column
  df.insert(loc=len(df.columns), column="totalcol", value=total_column)

  return df

In [ ]:
# Given a list of dataframes, remove header rows and keep only numeric rows
def prune_DA_dfs(dfs):

  print("calling prune_DA_dfs()")

  integer_regex = '[0-9]+' # Regular expression for integers

  pattern = r'(?i)total\s+for[eign]+ers?'
  foreigner_regex = re.compile(pattern, re.IGNORECASE)

  pruned_dfs = []

  # Loops through each page except the last
  # Uses regular expression to check for a header row
  # Modifies accordingly and resets the column names to avoid append complications
  # adds it to a list to be appended
  for i in range(len(dfs)-2):
    df = dfs[i]
    row_0 = df.iloc[0, 0] # First row, first column

    # Converting data to string type to allow use of regular expressions
    if hasattr(row_0, 'decode'):
      row_0 = row_0.decode('utf-8', errors='ignore')
    else:
      row_0 = str(row_0)
    try:
      matched_regex = re.search(integer_regex, row_0)
    except:
      print("could not use regular expressions in prune_DA_dfs()")

    # First row is data, extract the whole thing
    if matched_regex:
      new_df = df
    # First row is a header row, extract everything except it
    else:
      new_df = df[1:]

    new_df.columns = range(len(new_df.columns))
    pruned_dfs.append(new_df)

  ##### Handling second last page #######
  df = dfs[-2]
  row_0 = df.iloc[0, 0] # First row, first column

  # Converting data to string type to allow use of regular expressions
  if hasattr(row_0, 'decode'):
    row_0 = row_0.decode('utf-8', errors='ignore')
  else:
    row_0 = str(row_0)
  try:
    matched_regex = re.search(integer_regex, row_0)
  except:
    print("could not use regular expressions in prune_DA_dfs() - first row")

  print("handling last page")
  # First row is a data row
  if matched_regex:
    print("first row is a data row")
    new_df = df
  # First row is a header row, extract everything except it
  else:
    print("first row is not a data row")
    new_df = df[1:]

  # Check if there is a 'total foreigner row' (in the second last row, first column or last row, first column)
  if new_df.shape[0] >= 2:
    total_foreigner_row = new_df.iloc[-2, 0]
    total_foreigner_row2 = new_df.iloc[-1, 0]
    #print(total_foreigner_row)

    # Converting data to string type to allow use of regular expressions
    if hasattr(total_foreigner_row, 'decode'):
      total_foreigner_row = total_foreigner_row.decode('utf-8', errors='ignore')
    else:
      total_foreigner_row = str(total_foreigner_row)
    try:
      matched_regex_for = re.search(foreigner_regex, total_foreigner_row)
    except:
      print("could not use regular expressions in prune_DA_dfs() - total foreigners row")

    if hasattr(total_foreigner_row2, 'decode'):
      total_foreigner_row2 = total_foreigner_row2.decode('utf-8', errors='ignore')
    else:
      total_foreigner_row2 = str(total_foreigner_row2)
    try:
      matched_regex_for2 = re.search(foreigner_regex, total_foreigner_row2)
    except:
      print("could not use regular expressions in prune_DA_dfs() - total foreigners row")

    print("made it before checking for total foreigners row")
    # If there is a total foreigner row, remove it:
    if matched_regex_for:
      new_df = new_df.drop(new_df.index[-2])
      print("dropped the total foreigner row in the second last row")
    elif matched_regex_for2:
      new_df = new_df.drop(new_df.index[-1])
      print("dropped the total foreigner row in the last row")

  new_df.columns = range(len(new_df.columns))
  pruned_dfs.append(new_df)

  ##### Handling the last page #######
  # Do not grab the last row (will manually calculate ourselves)
  df = dfs[-1]
  row_0 = df.iloc[0, 0] # First row, first column


  # Converting data to string type to allow use of regular expressions
  if hasattr(row_0, 'decode'):
    row_0 = row_0.decode('utf-8', errors='ignore')
  else:
    row_0 = str(row_0)
  try:
    matched_regex = re.search(integer_regex, row_0)
  except:
    print("could not use regular expressions in prune_DA_dfs() - first row")

  print("handling last page")
  # First row is a data row, extract the whole thing except the last row
  if matched_regex:
    print("first row is a data row")
    new_df = df[:-1]
  # First row is a header row, extract everything except it and the last row
  else:
    print("first row is not a data row")
    new_df = df[1:-1]

  # Check if there is a 'total foreigner row' (in the second last row, first column)
  if new_df.shape[0] >= 2:
    total_foreigner_row = new_df.iloc[-2, 0]
    #print(total_foreigner_row)

    # Converting data to string type to allow use of regular expressions
    if hasattr(total_foreigner_row, 'decode'):
      total_foreigner_row = total_foreigner_row.decode('utf-8', errors='ignore')
    else:
      total_foreigner_row = str(total_foreigner_row)
    try:
      matched_regex_for = re.search(foreigner_regex, total_foreigner_row)
    except:
      print("could not use regular expressions in prune_DA_dfs() - total foreigners row")

    print("made it before checking for total foreigners row")
    # If there is a total foreigner row, remove it:
    if matched_regex_for:
      new_df = new_df.drop(new_df.index[-2])
      print("dropped the total foreigner row")

  print("made it past checking for total foreigners row")

  new_df.columns = range(len(new_df.columns))
  pruned_dfs.append(new_df)

  print("prune_last_page() has finished running")
  return pruned_dfs



In [ ]:
def calculate_DA_total(dfs):
  print("calling calculate_DA_total()")

  # Concatenate the pages together to calculate the sum (total row)

  temp_df = pd.concat(dfs, ignore_index=True)

  last_row_list = ["", "Total"]

  # Now calculate the sum:
  for i in range(len(temp_df.columns)):
    # Third column or beyond (first column is id, second is 'total)
    if i >= 2:
      col_numeric = pd.to_numeric(temp_df.iloc[:, i], errors='coerce')
      col_sum = col_numeric.sum()
      floored_col_sum = np.floor(col_sum)
      rounded_col_sum = int(floored_col_sum)
      last_row_list.append(rounded_col_sum)

  # Convert list to a pandas dataframe
  some_df = dfs[0]
  last_row_df = pd.DataFrame([last_row_list], columns=range(len(some_df.columns)))

  print(last_row_df)
  return last_row_df


In [ ]:
# Give a dataframe that has 5 columns
# Reconstruct the data frame to have 10 columns
# Filling in the missing columns with '.' (for compatible use with STATA)
def handle_5_col_form(df):
  print("called handle_5_col_form()", df.shape[1])

  header_row_df = pd.DataFrame([df.columns.tolist()], columns=df.columns)
  df_combined = pd.concat([header_row_df, df], ignore_index=True)

  df_combined.insert(loc=2, column='Male', value = ".")
  df_combined.insert(loc=3, column='Female', value = ".")
  df_combined.insert(loc=4, column='Other', value = ".")

  df_combined.insert(loc=6, column='Male2', value = ".")
  df_combined.insert(loc=7, column='Female2', value = ".")
  df_combined.insert(loc=8, column='Other2', value = ".")

  # Drop the last column
  df_combined = df_combined.drop(columns=df_combined.columns[-1])

  return df_combined


In [ ]:
def runocr(file):
    """Run OCR on a PDF file, replace it and rename with OCR_ prefix"""

    # Extract directory and filename
    directory = os.path.dirname(file)
    original_filename = os.path.basename(file)

    # Create new filename with OCR_ prefix
    new_filename = "OCR_" + original_filename
    new_file_path = os.path.join(directory, new_filename)

    print(f"Processing: {original_filename}")
    print(f"Will rename to: {new_filename}")

    # Run OCR to temporary file first
    temp_file = "temp_ocr_output.pdf"
    result = subprocess.run(
        ['ocrmypdf', '--force-ocr', file, temp_file],
        capture_output=True,
        text=True
    )

    if result.returncode == 0:
        # Remove original file
        os.remove(file)
        # Rename temporary file to new name
        shutil.move(temp_file, new_file_path)
        print(f"✓ OCR successful! Renamed to: {new_filename}")

        # Open and return the renamed file
        return_file = fitz.open(new_file_path)
        return return_file
    else:
        print("✗ OCR failed! Return code:", result.returncode)
        if result.stderr:
            print("Error:", result.stderr)
        # Clean up temporary file if it exists
        if os.path.exists(temp_file):
            os.remove(temp_file)
        return None

In [ ]:
def get_DA_dfs(folder_path):

  print("calling get_DA_dfs()")

  DA_forms = []
  failed_files = []

  # Gets all the files in a given folder
  files = os.listdir(folder_path)

  # If folder is not empty
  if files:
    # i now acts as the file index in the folder
    for i in range(len(files)):
      print("Processing file:", files[i])

      try:
        append_list = []

        file = os.path.join(folder_path, files[i])
        print("in try statement working on:", file)

        extracted = extract_DA_forms(file)

        print("extracted successfully")

        my_dfs = []

        for df in extracted:
          new_df = remove_empty_columns(df)
          if new_df.shape[1] == 9:
            fixed_df = add_missing_total_col(new_df)
          elif new_df.shape[1] == 5:
            fixed_df = handle_5_col_form(new_df)
          else:
            fixed_df = new_df
          my_dfs.append(fixed_df)

        first_row = get_DA_first_row(my_dfs[0])
        second_row = get_DA_second_row(my_dfs[0])

        append_list.append(first_row)
        append_list.append(second_row)

        pruned = prune_DA_dfs(my_dfs)

        concat_df = pd.concat(pruned, ignore_index=True)
        append_list.append(concat_df)

        total_row = calculate_DA_total(pruned)
        append_list.append(total_row)

        print("made it before resetting column names")

        for df in append_list:
          df.columns = range(len(df.columns))

        concat_df = pd.concat(append_list, ignore_index=True)

        print("made it before adding time stamp")

        # Add a time stamp column (to the leftmost column)
        temp_file_name = files[i].split('.')
        timestamp_val = temp_file_name[0]

        if i == 0:
          print("first page, need to fix timestamp")
          timestamp_col = ["", ""]
          for _ in range(concat_df.shape[0] - 2):
            timestamp_col.append(timestamp_val)
          print(len(timestamp_col))
          print(len(concat_df))
          concat_df.insert(loc=0, column='Timestamp', value=timestamp_col)
        else:
          concat_df.insert(loc=0, column='Timestamp', value=timestamp_val)

        print("made it after inserting time stamp ")

        # If not the first file, delete the header rows (first two rows)
        if i > 0:
          concat_df = concat_df.iloc[2:]

        DA_forms.append(concat_df)

        print("file done")

      except:
        print("Error occured on file:", files[i])
        failed_files.append(files[i])

  # Append files
  final_DA_df = pd.concat(DA_forms, ignore_index=True)

  # Export file to csv
  export_file_to_csv(final_DA_df, "Arrival_Departure")

  print("Failed files:", failed_files)


Testing

In [ ]:
# Official test
folder_path = '/content/drive/My Drive/PDF_Text_Extraction_Project/Arrival_Departure_PDFs'

get_DA_dfs(folder_path)

calling get_DA_dfs()
Processing file: 2018_04.pdf
in try statement working on: /content/drive/My Drive/PDF_Text_Extraction_Project/Arrival_Departure_PDFs/2018_04.pdf
extracted successfully
called remove_empty_columns()
called remove_empty_columns()
called remove_empty_columns()
calling prune_DA_dfs()
handling last page
first row is not a data row
made it before checking for total foreigners row
handling last page
first row is not a data row
made it before checking for total foreigners row
dropped the total foreigner row
made it past checking for total foreigners row
prune_last_page() has finished running
calling calculate_DA_total()
  0      1      2       3     4       5      6       7     8       9
0    Total  55762  128113  2765  186701  65876  143608  1724  211208
made it before resetting column names
made it before adding time stamp
first page, need to fix timestamp
119
119
made it after inserting time stamp 
file done
Processing file: 2025_07.pdf
in try statement working on: /con

In [ ]:
#folder_path = '/content/drive/My Drive/PDF_Text_Extraction_Project/Arrival_Departure_PDFs/2020_01.pdf'

folder_path = '/content/drive/My Drive/PDF_Text_Extraction_Project/Problematic_PDFS/OCR_Copy of 2024_09.pdf'

test_dfs = extract_DA_forms(folder_path)
print(len(test_dfs))

for i in range(len(test_dfs)):
  export_file_to_csv(test_dfs[i], f"test_2024_09{i}")

0
